In [2]:
%cd ~/contracode
import numpy as np
import pickle
import gzip
from tqdm.auto import tqdm
import pandas as pd
import time
from typing import Iterable
from loguru import logger
import multiprocessing as mp
# import modin.pandas as pd
import torch
import torch.nn as nn
from torch.utils.data.dataset import Dataset
from transformers import BertTokenizerFast
from representjs import DATA_DIR
from tqdm import tqdm
# import swifter

/home/ajay/contracode


# Shard the train set for tokenization

In [ ]:
# train_df = load_data("/data/ajay/contracode/data/hf_data/augmented_pretrain_df.train.pickle.gz")

In [33]:
from multiprocessing.pool import ThreadPool

def split_df(df, save_pattern, num_chunks=160):
    # Split data frame into chunks
    chunk_size = int(df.shape[0] / num_chunks)
    def save_data(data):
        chunk_i, start, chunk_size, df, save_pattern = data
        save_path = save_pattern.format(chunk_i)
        df_subset = df.iloc[start : start + chunk_size]
        df_subset.to_pickle(save_path)
        print("Saved ", save_path)
    items = [(i, start, chunk_size, df, save_pattern) for i, start in enumerate(range(0, df.shape[0], chunk_size))]
    with ThreadPool(64) as pool:
        pool.map(save_data, items)

In [34]:
chunk_dir = "/data/ajay/contracode/data/hf_data/train_chunks"
!mkdir -p {chunk_dir}
split_df(train_df, chunk_dir + "/augmented_pretrain_df.{:04d}.train.pickle.gz", 160)

Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0002.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0009.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0012.train.pickle.gz
Saved Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0006.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0022.train.pickle.gz
 /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0000.train.pickle.gzSaved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0019.train.pickle.gz

Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0004.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0026.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0016.train.pickle.gz
Saved  /data/ajay/co

Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0040.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0033.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0048.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0045.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0023.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0063.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0024.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0055.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0035.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0012.train.pickle.gz
Saved  /data/ajay/co

Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0101.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0105.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0106.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0100.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0102.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0118.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0122.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0110.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0109.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0123.train.pickle.gz
Saved  /data/ajay/co

In [43]:
tqdm.pandas()

path = "/data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0000.train.pickle.gz"

def load_tokenizer(path="data/vocab/8k_bpe/8k_bpe-vocab.txt"):
    return BertTokenizerFast(path, clean_text=True, lowercase=False, strip_accents=True, unk_token="<unk>")

def load_data(path):
    return pd.read_pickle(path)

tokenizer = load_tokenizer()
df_shard = load_data(path)
df_shard['toks'] = df_shard['text'].progress_apply(lambda x: np.asarray(tokenizer.encode(x)))
df_shard = df_shard[['data_idx', 'toks']]

/home/ajay/miniconda3/envs/paras/lib/python3.8/site-packages/tqdm/std.py:670: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 117298/117298 [02:27<00:00, 793.01it/s] 


In [52]:
from tqdm.contrib.concurrent import process_map

dfs = []

files = []
for i in tqdm(list(range(161))):
    path = f"/data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.{i:04d}.train.pickle.gz"
    files.append(path)

def load_file(fname):
    out = pd.read_pickle(fname)
    return out
    
dfs = process_map(load_file, files, max_workers=16)

100%|██████████| 161/161 [00:00<00:00, 6202.48it/s]


DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0000.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0009.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0003.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0005.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0008.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0006.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0013.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_to

DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0122.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0025.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0123.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0124.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0022.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0125.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0126.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_to

 /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0158.train.pickle.gz
DONE Loaded file DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0023.train.pickle.gz
 /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0160.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0159.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0008.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0010.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0044.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0005.t

In [58]:
merged_df = pd.concat(dfs)

In [61]:
merged_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18767791 entries, 0 to 18767790
Data columns (total 2 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   data_idx  int64 
 1   toks      object
dtypes: int64(1), object(1)
memory usage: 7.0 GB


In [ ]:
merged_df.to_pickle('/data/ajay/contracode/data/hf_data/merged_tok.pickle.gz')

In [1]:
merged_df

NameError: name 'merged_df' is not defined

# Repack data into plain pickle format

In [21]:
import pyarrow as pa
import pyarrow.feather as feather

%time test_df = pd.read_pickle('/data/ajay/contracode/data/hf_data/augmented_pretrain_df_tok.test.pickle.gz')
%time feather_test_df = pa.Table.from_pandas(test_df)
%time feather.write_feather(feather_test_df, '/data/ajay/contracode/data/hf_data/feather_tok/test_lz4.feather', compression='lz4')

CPU times: user 1.52 s, sys: 116 ms, total: 1.64 s
Wall time: 1.64 s
CPU times: user 648 ms, sys: 296 ms, total: 944 ms
Wall time: 940 ms
CPU times: user 528 ms, sys: 192 ms, total: 720 ms
Wall time: 761 ms


In [35]:
%time train_df = pd.read_pickle('/data/ajay/contracode/data/hf_data/merged_tok.pickle.gz')
%time feather_train_df = pa.Table.from_pandas(train_df)
%time feather.write_feather(feather_train_df, '/data/ajay/contracode/data/hf_data/feather_tok/train_lz4.feather', compression='lz4')

KeyboardInterrupt: 

KeyboardInterrupt: 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/ajay/miniconda3/envs/paras/lib/python3.8/site-packages/IPython/core/magics/execution.py", line 1313, in time
    out = eval(code, glob, local_ns)
  File "<timed eval>", line 1, in <module>
NameError: name 'feather_train_df' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ajay/miniconda3/envs/paras/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ajay/miniconda3/envs/paras/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/home/ajay/miniconda3/envs/paras/lib/python3.8/site-pa

TypeError: object of type 'NoneType' has no len()

In [55]:
chunk_size = int(len(train_df) / 10)
for i in tqdm(list(range(11))):
    %time sampled_train_df = train_df[chunk_size * i : chunk_size * i + chunk_size]
    %time feather_train_df = pa.Table.from_pandas(sampled_train_df)
    %time feather.write_feather(feather_train_df, f'/data/ajay/contracode/data/hf_data/feather_tok/train_lz4.feather.{i:02d}', compression='lz4')

  0%|          | 0/11 [00:00<?, ?it/s]

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 314 µs
CPU times: user 10.3 s, sys: 3.04 s, total: 13.3 s
Wall time: 13.2 s


  9%|▉         | 1/11 [00:23<03:55, 23.54s/it]

CPU times: user 8.61 s, sys: 1.74 s, total: 10.4 s
Wall time: 10.3 s
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 590 µs
CPU times: user 10.7 s, sys: 3.72 s, total: 14.4 s
Wall time: 14.3 s


 18%|█▊        | 2/11 [00:47<03:34, 23.81s/it]

...

CPU times: user 9.9 s, sys: 2.48 s, total: 12.4 s
Wall time: 12 s
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 659 µs
CPU times: user 10.7 s, sys: 4.12 s, total: 14.9 s
Wall time: 14.8 s


 82%|████████▏ | 9/11 [03:57<00:54, 27.09s/it]

CPU times: user 9.21 s, sys: 3.07 s, total: 12.3 s
Wall time: 12 s
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 406 µs
CPU times: user 10.8 s, sys: 4.46 s, total: 15.2 s
Wall time: 15.1 s


100%|██████████| 11/11 [04:25<00:00, 24.17s/it]

CPU times: user 9.46 s, sys: 3.89 s, total: 13.3 s
Wall time: 13 s
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 567 µs
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 912 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 913 µs


In [44]:
%time feather.write_feather(feather_train_df, '/data/ajay/contracode/data/hf_data/feather_tok/train_lz4.feather', compression='lz4')

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 501 µs


In [50]:
int(len(train_df) * 0.1)
len(train_df[int(len(train_df) * 0.1):])

16891012

In [51]:
len(feather_train_df)

0

In [56]:
%time feather.read_feather('/data/ajay/contracode/data/hf_data/feather_tok/train_lz4.feather.00')

CPU times: user 4.69 s, sys: 6.58 s, total: 11.3 s
Wall time: 11.3 s


,data_idx,toks
0,1319387,"[6, 1391, 98, 39, 1302, 18, 94, 19, 101, 1305,..."
1,1290376,"[6, 1302, 98, 18, 2110, 22, 1374, 19, 101, 130..."
2,1759800,"[6, 1391, 98, 39, 1302, 18, 1666, 22, 1415, 22..."
3,1829150,"[6, 1391, 98, 39, 1302, 18, 77, 22, 2536, 19, ..."
4,293100,"[6, 1391, 98, 39, 1302, 18, 19, 101, 1308, 92,..."
...,...,...
1876774,404761,"[6, 1302, 98, 18, 14, 1568, 22, 1666, 22, 1397..."
1876775,1014435,"[6, 1302, 98, 18, 79, 19, 101, 1308, 1586, 158..."
1876776,1429626,"[6, 1302, 98, 18, 1732, 19, 101, 1308, 1567, 3..."
1876777,261604,"[6, 1302, 98, 18, 19, 101, 1735, 1038, 1383, 1..."


In [64]:
import pyarrow.feather as feather
from tqdm.contrib.concurrent import thread_map

files = [f'/data/ajay/contracode/data/hf_data/feather_tok/train_lz4.feather.{i:02d}' for i in range(11)]
%time dfs = thread_map(feather.read_feather, files, max_workers=16)


CPU times: user 53.5 s, sys: 1min 31s, total: 2min 24s
Wall time: 2min 20s


In [66]:
import pandas as pd
files = [f'/data/ajay/contracode/data/hf_data/feather_tok/train_lz4.feather.{i:02d}' for i in range(11)]
%time dfs = thread_map(pd.read_feather, files, max_workers=16)


CPU times: user 57.4 s, sys: 1min 27s, total: 2min 25s
Wall time: 2min 19s


In [75]:
import glob
from pathlib import Path
glob.glob('/data/ajay/contracode/data/hf_data/feather_tok/train_lz4.feather' + '*')

['/data/ajay/contracode/data/hf_data/feather_tok/train_lz4.feather.10',
 '/data/ajay/contracode/data/hf_data/feather_tok/train_lz4.feather.09',
 '/data/ajay/contracode/data/hf_data/feather_tok/train_lz4.feather.02',
 '/data/ajay/contracode/data/hf_data/feather_tok/train_lz4.feather.07',
 '/data/ajay/contracode/data/hf_data/feather_tok/train_lz4.feather.06',
 '/data/ajay/contracode/data/hf_data/feather_tok/train_lz4.feather.03',
 '/data/ajay/contracode/data/hf_data/feather_tok/train_lz4.feather.04',
 '/data/ajay/contracode/data/hf_data/feather_tok/train_lz4.feather.00',
 '/data/ajay/contracode/data/hf_data/feather_tok/train_lz4.feather.08',
 '/data/ajay/contracode/data/hf_data/feather_tok/train_lz4.feather.01',
 '/data/ajay/contracode/data/hf_data/feather_tok/train_lz4.feather.05']